In [2]:
from Sindonews import Sindonews #class news Scrapper
from datetime import timedelta, date, datetime
from IPython.display import clear_output

In [3]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [4]:
obj = Sindonews()
end_date = date(2018, 7, 24)
start_date = date(2018, 1, 16)
init_page = 34
init_offset = 0

In [5]:
for single_date in daterange(start_date, end_date):
    clear_output()
    print(single_date.strftime("%Y-%m-%d"))
    obj.getAllBerita([], init_page, 0, init_offset, single_date.strftime("%Y-%m-%d"))
    init_page = 1

2018-01-16
page  5
https://index.sindonews.com/index/0/0?t=2018-01-16
https://ekbis.sindonews.com/read/1274218/34/90-pengusaha-muslim-di-yogyakarta-terjebak-utang-dan-riba-1516115977
Insert berita  90% Pengusaha Muslim di Yogyakarta Terjebak Utang dan Riba
salah2
https://daerah.sindonews.com/read/1274231/194/lanal-batam-amankan-boat-pancung-angkut-narkoba-1516120364
Insert berita  Lanal Batam Amankan Boat Pancung Angkut Narkoba
salah2
https://metro.sindonews.com/read/1274155/171/operasi-lintas-jaya-penegakan-hukum-harus-berbasis-elektronik-1516104324
Insert berita  Operasi Lintas Jaya, Penegakan Hukum Harus Berbasis Elektronik
salah2
https://sports.sindonews.com/read/1274230/51/irwasum-target-pgn-popsivo-polwan-juara-proliga-2018-1516119800
Insert berita  Irwasum Target PGN Popsivo Polwan Juara Proliga 2018
salah2
https://sports.sindonews.com/read/1274228/50/parker-tetap-ngotot-sanggup-robohkan-joshua-1516119422
Insert berita  Parker Tetap Ngotot Sanggup Robohkan Joshua
salah2
https://

salah2
https://soccer.sindonews.com/read/1274195/58/lawan-psms-medan-persib-masih-harus-kerja-keras-1516110513
Insert berita  Lawan PSMS Medan, Persib Masih Harus Kerja Keras
salah2
https://sports.sindonews.com/read/1274194/49/setelah-delapan-tahun-pensiun-kubica-kembali-ke-formula-1-1516110022
Insert berita  Setelah Delapan Tahun Pensiun, Kubica Kembali ke Formula 1
salah2
https://international.sindonews.com/read/1274179/43/iran-rencana-as-di-suriah-hanya-akan-timbulkan-perang-baru-1516108134
Insert berita  Iran: Rencana AS di Suriah Hanya Akan Timbulkan Perang Baru
salah2
https://metro.sindonews.com/read/1274193/171/resmikan-rsia-menkes-harap-pasien-tertangani-dengan-baik-1516109736
Insert berita  Resmikan RSIA, Menkes Harap Pasien Tertangani dengan Baik
salah2
https://ekbis.sindonews.com/read/1274182/34/gudang-bulog-palopo-pastikan-stok-beras-luwu-raya-aman-1516108302
Insert berita  Gudang Bulog Palopo Pastikan Stok Beras Luwu Raya Aman
salah2
https://soccer.sindonews.com/read/12741

salah2
https://sports.sindonews.com/read/1274146/50/garcia-sudah-lupakan-kekalahan-profesional-pertamanya-1516103184
Insert berita  Garcia Sudah Lupakan Kekalahan Profesional Pertamanya
salah2
https://sports.sindonews.com/read/1274145/51/bhayangkara-samator-bidik-gelar-juara-proliga-2018-1516102897
Insert berita  Bhayangkara Samator Bidik Gelar Juara Proliga 2018
salah2
https://international.sindonews.com/read/1274142/43/qatar-bantah-jet-tempurnya-cegat-2-pesawat-sipil-uea-1516102452
Insert berita  Qatar Bantah Jet Tempurnya Cegat 2 Pesawat Sipil UEA
salah2
https://lifestyle.sindonews.com/read/1274080/157/ini-harga-tiket-konser-celine-dion-di-indonesia-1516092932
Insert berita  Ini Harga Tiket Konser Celine Dion di Indonesia
salah2
https://ekbis.sindonews.com/read/1274141/34/bulog-siap-salurkan-142-ribu-ton-bansos-rastra-bulan-ini-1516102131
Insert berita  Bulog Siap Salurkan 142 Ribu Ton Bansos Rastra Bulan Ini
salah2
https://metro.sindonews.com/read/1274140/171/partai-perindo-dukung-

salah2
https://lifestyle.sindonews.com/read/1274048/187/persiapan-nikah-taeyang-big-bang-beli-rumah-mewah-rp4-8-miliar-1516087248
Insert berita  Persiapan Nikah, Taeyang Big Bang Beli Rumah Mewah Rp4,8 Miliar
salah2
https://daerah.sindonews.com/read/1274097/174/berkonsep-semimodern-pemkab-klungkung-revitalisasi-pasar-tradisional-1516094884
Insert berita  Berkonsep Semimodern, Pemkab Klungkung Revitalisasi Pasar Tradisional
salah2
https://daerah.sindonews.com/read/1274096/22/anggaran-perjalanan-dinas-dprd-salatiga-tembus-rp12-9-miliar-1516094817
Insert berita  Anggaran Perjalanan Dinas DPRD Salatiga Tembus Rp12,9 Miliar
salah2
https://ekbis.sindonews.com/read/1274094/34/mekanisme-diubah-bansos-rastra-siap-disalurkan-mulai-bulan-ini-1516094580
Insert berita  Mekanisme Diubah, Bansos Rastra Siap Disalurkan Mulai Bulan Ini
salah2
page  18
https://index.sindonews.com/index/0/130?t=2018-01-16
https://nasional.sindonews.com/read/1274095/12/instruksi-airlangga-golkar-segera-akhiri-masa-tugas-d

https://ekbis.sindonews.com/read/1274050/34/puan-undang-dua-menteri-hingga-bankir-bahas-bansos-pangan-1516087730
Insert berita  Puan Undang Dua Menteri hingga Bankir Bahas Bansos Pangan
salah2
https://metro.sindonews.com/read/1274049/170/jenguk-korban-balkon-bei-ambruk-menristek-dikti-saya-prihatin-1516087516
Insert berita  Jenguk Korban Balkon BEI Ambruk, Menristek Dikti: Saya Prihatin
salah2
https://sports.sindonews.com/read/1274047/49/dovizioso-yakin-juara-dengan-motor-baru-1516087000
Insert berita  Dovizioso Yakin Juara dengan Motor Baru
salah2
page  22
https://index.sindonews.com/index/0/170?t=2018-01-16
https://metro.sindonews.com/read/1274045/170/ciptakan-tertib-lalin-gubernur-dki-buka-operasi-lintas-jaya-2018-1516086563
Insert berita  Ciptakan Tertib Lalin, Gubernur DKI Buka Operasi Lintas Jaya 2018
salah2
https://ekbis.sindonews.com/read/1274042/34/jasa-raharja-cairkan-dana-santuan-senilai-rp414-miliar-1516086266
Insert berita  Jasa Raharja Cairkan Dana Santuan Senilai Rp414 M

salah2
page  26
https://index.sindonews.com/index/0/210?t=2018-01-16
https://ekbis.sindonews.com/read/1274001/35/airbus-beri-sinyal-hentikan-produksi-pesawat-superjumbo-a380-1516078195
Insert berita  Airbus Beri Sinyal Hentikan Produksi Pesawat Superjumbo A380
salah2
https://lifestyle.sindonews.com/read/1273982/155/studi-bawang-merah-tak-lagi-perih-di-mata-saat-diiris-1516075161
Insert berita  Studi: Bawang Merah Tak Lagi Perih di Mata Saat Diiris
salah2
https://metro.sindonews.com/read/1274006/170/museum-bahari-gudang-rempah-voc-dan-jejak-maritim-nusantara-1516078846
Insert berita  Museum Bahari, Gudang Rempah VOC dan Jejak Maritim Nusantara
salah2
https://soccer.sindonews.com/read/1274005/56/perbaiki-pertahanan-inter-milan-rekrut-bek-argentina-1516078821
Insert berita  Perbaiki Pertahanan, Inter Milan Rekrut Bek Argentina
salah2
https://autotekno.sindonews.com/read/1274004/122/dji-kenalkan-gimbal-baru-di-ces-2018-1516078678
Insert berita  DJI Kenalkan Gimbal Baru di CES 2018
salah2
h

masuk
https://ekbis.sindonews.com/read/1273958/32/melantai-di-bei-saham-lck-global-kedaton-dibuka-naik-104-poin-1516068777
Insert berita  Melantai di BEI, Saham LCK Global Kedaton Dibuka Naik 104 Poin
masuk
https://international.sindonews.com/read/1273962/43/memanas-jet-tempur-qatar-dituduh-cegat-2-pesawat-sipil-uea-1516069450
Insert berita  Memanas, Jet Tempur Qatar Dituduh Cegat 2 Pesawat Sipil UEA
masuk
https://daerah.sindonews.com/read/1273961/22/sudirman-said-galang-kekuatan-5-000-mantan-prajurit-di-pilgub-jateng-1516069306
Insert berita  Sudirman Said Galang Kekuatan 5.000 Mantan Prajurit di Pilgub Jateng
masuk
https://nasional.sindonews.com/read/1273960/12/konflik-hanura-tanda-parpol-belum-siap-hadapi-pilkada-serentak-1516069036
Insert berita  Konflik Hanura, Tanda Parpol Belum Siap Hadapi Pilkada Serentak
masuk
https://ekbis.sindonews.com/read/1273959/32/ihsg-dibuka-bersinar-meski-bursa-saham-asia-memerah-1516068974
Insert berita  IHSG Dibuka Bersinar meski Bursa Saham Asia Mem

SSLError: HTTPSConnectionPool(host='ekbis.sindonews.com', port=443): Max retries exceeded with url: /read/1274103/34/potensi-besar-pemerintah-didorong-perkuat-pertanian-kelapa-1516095659 (Caused by SSLError(SSLError("bad handshake: SysCallError(10060, 'WSAETIMEDOUT')",),))